In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
%load_ext autoreload
%autoreload 2

import logging

logging.basicConfig(level=logging.INFO)

from abc import ABC, abstractmethod
from contextlib import AbstractContextManager
from typing import Protocol, runtime_checkable

import torch
from torch import Tensor, jit, nn, tensor
from torch.linalg import matrix_norm

# from linodenet.lib import singular_triplet
from torchinfo import summary

# from linodenet.parametrize import Parametrization, SpectralNormalization

In [ ]:
import torch
from torch import jit, Tensor
from typing import Final

In [ ]:
EPS = {
    torch.bfloat16: 2**-8,
    torch.float16: 2**-11,
    torch.float32: 2**-24,
    torch.float64: 2**-53,
}


@jit.script
def relative_error(xhat: Tensor, x_true: Tensor) -> Tensor:
    eps: float = {
        torch.bfloat16: 2**-8,
        torch.float16: 2**-11,
        torch.float32: 2**-24,
        torch.float64: 2**-53,
    }[xhat.dtype]
    return torch.abs(xhat - x_true) / (torch.abs(x_true) + eps)


relative_error(torch.randn(5), torch.randn(5))

In [ ]:
torch.randn(5).abs()

In [ ]:
import linodenet

In [ ]:
B, M, N = 4, 3, 3
x = torch.randn(B, M)

# setup reference model
reference_model = nn.Linear(M, N, bias=False)
symmetrized_weight = symmetric(reference_model.weight)
reference_model.weight = nn.Parameter(symmetrized_weight)
assert is_symmetric(reference_model.weight)

# setup vanilla model
model = nn.Linear(M, N, bias=False)
with torch.no_grad():
    model.weight.copy_(reference_model.weight)

# check compatibility
check_model(model, input_args=(x,), reference_model=reference_model, test_jit=True)

# now, parametrize
weight = model.weight
param = Parametrize(weight, symmetric)
param.zero_grad(set_to_none=True)
model.weight = param.parametrized_tensor
model.param = param

# check compatibility
check_model(model, input_args=(x,), reference_model=reference_model, test_jit=True)

In [ ]:
m, n = 5, 5
tensor = torch.randn(m, n)
weight = nn.Parameter(tensor)
param = Parametrize(weight, symmetric)
param.zero_grad(set_to_none=True)

In [ ]:
model = nn.Linear(m, n)
weight = model.weight
param = Parametrize(weight, symmetric)
param.zero_grad(set_to_none=True)
model.weight = param.parametrized_tensor
model.param = param
summary(model)

In [ ]:
model = nn.Linear(m, n, bias=False)
with torch.no_grad():
    model.weight.copy_(reference_model.weight)

In [ ]:
x = torch.randn(7, m)
check_model(model, input_args=(x,), reference_model=reference_model, test_jit=True)

## now, parametrize

In [ ]:
im

In [ ]:
weight = model.weight
param = Parametrize(weight, symmetric)
param.zero_grad(set_to_none=True)
model.weight = param.parametrized_tensor
model.param = param
check_model(model, input_args=(x,), reference_model=reference_model, test_jit=True)